In [157]:
import time
import pandas as pd
import numpy as np

d1 = pd.read_csv("data/材料1.csv")
d2 = pd.read_csv("data/材料2.csv")
d3 = pd.read_csv("data/材料3.csv")
d4 = pd.read_csv("data/材料4.csv")

d1['材料'] = 1
d2['材料'] = 2
d3['材料'] = 3
d4['材料'] = 4

pd.set_option('future.no_silent_downcasting', True)

data = pd.concat([d1, d2, d3, d4])
data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})

B_ = data.iloc[:, 4:-1].values

lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('材料')
data.columns = lst

# std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(B_, axis=1)  # 峰值
min_flux = np.min(B_, axis=1)
delta_flux = max_flux - min_flux
sqrt_root = np.mean(B_, axis=1)  # 均方根
ave_flux = np.mean(B_, axis=1)
sqrt_ave = sqrt_root / ave_flux

data['B_max'] = max_flux
data['B_min'] = min_flux
data['delta_B'] = delta_flux
data['B_rms'] = sqrt_root
data['B_ave'] = ave_flux
data['k_f'] = sqrt_ave

filter_data = data[
    ['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min', 'delta_B', 'B_rms', 'B_ave', 'k_f', '磁芯损耗']
]

# to = pd.DataFrame(filter_data)
# to.to_csv("~/Desktop/data.csv", index=False)
filter_data.head()

,温度,频率,励磁波形,材料,B_max,B_min,delta_B,B_rms,B_ave,k_f,磁芯损耗
0,25,50030,1,1,0.028849,-0.028840,0.057689,-4.003908e-11,-4.003908e-11,1.0,1997.955250
1,25,50020,1,1,0.031419,-0.031427,0.062846,-2.539062e-11,-2.539062e-11,1.0,2427.749830
2,25,50020,1,1,0.035535,-0.035513,0.071047,-9.765699e-13,-9.765699e-13,1.0,3332.725760
3,25,50020,1,1,0.040015,-0.040025,0.080041,-5.859375e-11,-5.859375e-11,1.0,4502.908007
4,25,50030,1,1,0.045028,-0.045085,0.090113,4.101562e-11,4.101562e-11,1.0,6063.023248


In [158]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

start_time = time.time()
X = data[['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min']]  # 自变量 
y = data['磁芯损耗']  # 因变量

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ExtraTreesRegressor(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
end_time = time.time()

mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
mse = mean_squared_error(y_test, y_pred)

print(f"mape: {mape:.2f}, mse: {mse:.2f}")

mape: 9.16, mse: 1365179387.41


In [159]:
w1, w2 = 1, 1


def optim_target(t):
    T = t[0]
    f = t[1]
    wire = t[2]
    material = t[3]
    B_max = t[4]
    B_min = t[5]

    x = pd.DataFrame([[T, f, wire, material, B_max, B_min]],
                     columns=['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min'])
    y_pred = model.predict(x)[0]
    if y_pred < 0:
        return 1e9
    # res = w1 * y_pred - w2 * B_max * f
    # return res
    return y_pred

In [160]:
from pyswarm import pso

In [166]:
min_up = np.max(data['B_min'])
min_low = np.min(data['B_min'])

max_up = np.max(data['B_max'])
max_low = np.min(data['B_max'])

f_up = np.max(data['频率'])
f_low = np.min(data['频率'])

# print("B_min: ", min_up, min_low)
# print("B_max: ", max_up, max_low)
# print("频率:  ", f_up, f_low)

# lb = [25, 50000, 1, 1, 0.01, -0.1]
# ub = [90, 500000, 3, 4, 0.1, -0.01] 

lb = [25, f_low, 1, 1, max_low, min_low]
ub = [90, f_up, 3, 4, max_up, min_up]

part1 = 1e7
part2 = -1e7

t_param = None
cur = -1

start = time.time()
for i in range(1):
    # Running the PSO optimization
    best_params, best_score = pso(
        optim_target,
        lb,
        ub,
        swarmsize=50,  # 适当减小粒子群规模
        maxiter=50  # 减少最大迭代次数
    )

    print(f"Epoch {i + 1}:")
    print(f"\tBest Parameters: {best_params}")
    print(f"\tBest Score (Objective Function Value): {best_score}\n")

    res_f = best_params[1]
    res_b_max = best_params[4]

    x = pd.DataFrame([best_params], columns=['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min'])

    # X = data[['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min']]  # 自变量 

    f1 = model.predict(x)

    # f2 = res_f * res_b_max

    # t = f1 - f2
    # print(f"f1: {f1}")
    # print(f"f2: {res_f * res_b_max}")
    # if (f1 < part1 or f2 > part2) and t < cur:
    #     part1 = f1
    #     part2 = f2
    #     t = cur
    #     t_param = best_params
    
    if f1 < part1:
        part1 = f1
        t_param = best_params
        

end = time.time()

# print(f"\n\nBest f1: {part1}\t\tBest f2: {part2}")
# print(f"Time taken: {end - start}")

print(part1)
print(t_param)

print(t_param[1] * t_param[-2])
# lb = [40, 100000, 1, 1, 0.02, -0.08]  # 更新后的下界，移除了励磁波形和材料
# ub = [70, 300000, 3, 4, 0.08, -0.02]  # 更新后的上界

Stopping search: maximum iterations reached --> 50
Epoch 1:
	Best Parameters: [ 9.00000000e+01  4.99900000e+04  1.00000000e+00  4.00000000e+00
  1.27224651e-02 -9.36323100e-03]
	Best Score (Objective Function Value): 483.3411502176052

[483.34115022]
[ 9.00000000e+01  4.99900000e+04  1.00000000e+00  4.00000000e+00
  1.27224651e-02 -9.36323100e-03]
635.9960326649954


In [162]:
# print(t_param)
label = ['温度', '频率', '励磁波形', '材料', 'B_max', 'B_min']
for i in range(len(label)):
    print(f"{label[i]}:  {t_param[i]}")

print(f"\nBest f1: {part1[0]}\t\tBest f2: {part2}")
# Best f1: [1518.55673455]      Best f2: 3323.2754858418803
# Best f1: [1001.54089263]	    Best f2: 2504.9969226530347
# Best f1: [1277.66594449]		Best f2: 3050.8233393661712 
#       [ 8.69463990e+01  1.58521354e+05  1.00000000e+00  1.36521605e+00  1.92455039e-02 -1.00000000e-02]

温度:  90.0
频率:  88762.03229358177
励磁波形:  1.0
材料:  2.4641384794294336
B_max:  0.012691223457035643
B_min:  -0.009363231

Best f1: 483.77720007390474		Best f2: -10000000.0


In [163]:
# 温度:  61.89416515106509
# 频率:  158685.74171416223
# 励磁波形:  3.0
# 材料:  3.9999968119434497
# B_max:  0.012426007598233561
# B_min:  -0.009363231
# 
# Best f1: 975.4730120122099		Best f2: 1971.8302322715083

In [164]:
"""
温度:  75.09368254920875
频率:  141252.31113367257
励磁波形:  1.0
材料:  1.144178393350389
B_max:  0.021811679514510438
B_min:  -0.009363231

Best 磁芯损耗: 1311.834675964497		Best 传输磁能: 3080.9501411315805
"""

'\n温度:  75.09368254920875\n频率:  141252.31113367257\n励磁波形:  1.0\n材料:  1.144178393350389\nB_max:  0.021811679514510438\nB_min:  -0.009363231\n\nBest 磁芯损耗: 1311.834675964497\t\tBest 传输磁能: 3080.9501411315805\n'

In [165]:
# 温度:  72.2521174735506
# 频率:  158521.3580036507
# 励磁波形:  1.000117723503394
# 材料:  3.308706484613077
# B_max:  0.01924550614866515
# B_min:  -0.009363231
# 
# Best f1: 1403.3749871990028		Best f2: 3050.823770154009